In [1]:
import pandas as pd 
datas_py = pd.read_csv("../src/data/train.csv")

In [2]:
datas_py.head() #1 458 645 lignes

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [3]:
try:
    import pyspark
except:
    import findspark
    findspark.init()
    import pyspark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('taxi_trip').getOrCreate()

In [5]:
df = spark \
    .read \
    .format("csv") \
    .option("header", "true") \
    .load("../src/data/train.csv")

# Vitesse de chaque trajet 

J'utilise ici la formule de Haversine pour cacluler la distance en km entre un point A et un point B.
$$a = sin^2(\frac{lat_B-lat_A}{2})+ cos(lat_A)*cos(lat_B)*sin^2(\frac{long_B-long_A}{2})$$
$$d=2*r*arcsin(\sqrt{a})$$

avec r le rayon de la sphère sur laquelle sont placés les points. On considérera ici la terre comme sphérique, ce qui donne de bon résultat en pratique.
D'après la formule de wikipédia : https://en.wikipedia.org/wiki/Haversine_formula


On sait ensuite que :
$$vitesse = \frac{distance}{temps}$$

On aura obtiendra donc ici la vitesse moyenne du trajet en km/seconde, ce qui est une unité peu parlante mais qui permettra de comparer les trajets entre eux. On pourra par exemple voir qui sont les taxis ayant une vitesse moyenne plus rapide que les autres. Si nécessaire on peut ensuite adapter l'unité.

In [6]:
from math import radians,sin,cos
from pyspark.sql.functions import *

In [7]:
r = 6371
df_with_speed = df \
    .withColumn("a", pow(sin(radians(col("dropoff_latitude") - col("pickup_latitude")) / 2), 2) + cos(radians(col("pickup_latitude"))) * cos(radians(col("dropoff_latitude"))) * pow(sin(radians(col("dropoff_longitude") - col("pickup_longitude")) / 2), 2)) \
    .withColumn("distance", asin(sqrt(col("a"))) * 2 * r) \
    .withColumn("speed",col("distance") / col("trip_duration"))

In [8]:
df_with_speed.take(5)

[Row(id='id2875421', vendor_id='2', pickup_datetime='2016-03-14 17:24:55', dropoff_datetime='2016-03-14 17:32:30', passenger_count='1', pickup_longitude='-73.982154846191406', pickup_latitude='40.767936706542969', dropoff_longitude='-73.964630126953125', dropoff_latitude='40.765602111816406', store_and_fwd_flag='N', trip_duration='455', a=1.3830896636179652e-08, distance=1.498520779647477, speed=0.003293452262961488),
 Row(id='id2377394', vendor_id='1', pickup_datetime='2016-06-12 00:43:35', dropoff_datetime='2016-06-12 00:54:38', passenger_count='1', pickup_longitude='-73.980415344238281', pickup_latitude='40.738563537597656', dropoff_longitude='-73.999481201171875', dropoff_latitude='40.731151580810547', store_and_fwd_flag='N', trip_duration='663', a=2.0078128522717423e-08, distance=1.805507168795824, speed=0.002723238565302902),
 Row(id='id3858529', vendor_id='2', pickup_datetime='2016-01-19 11:35:24', dropoff_datetime='2016-01-19 12:10:48', passenger_count='1', pickup_longitude='-7

# Nombre de trajets effectués en fonction du jour de la semaine

In [9]:
df_with_day_of_week = df \
    .withColumn("pickup_datetime",to_timestamp(col("pickup_datetime"))) \
    .withColumn("week_day_number", date_format(col("pickup_datetime"), "u")) \
    .withColumn("week_day", date_format(col("pickup_datetime"), "E")) \

In [10]:
df_with_trajet_par_jour = df_with_day_of_week.rdd \
    .map(lambda x : (x.week_day,1)) \
    .reduceByKey(lambda x,y: x + y)

In [11]:
df_with_trajet_par_jour.collect()

[('Wed', 210136),
 ('Mon', 187418),
 ('Fri', 223533),
 ('Thu', 218574),
 ('Tue', 202749),
 ('Sun', 195366),
 ('Sat', 220868)]

On utilise ensuite la méthode du MapReduce pour calculer le nombre de trajet en fonction du jour de la semaine.
Pourquoi utiliser le MapReduce avec le `reduceByKey` et non pas le `groupByKey`.
Lorsque le montant de données est élévé, `groupByKey` est moins performant que `reduceByKey`. Voyons pourquoi.


Lorsqu'on utilise les RDD, chaque morceau de rdd est appelé partition. Chaque partition appartient à un executeur.
`groupByKey` va chercher à regrouper les clés identiques dans la même partitition pour ensuite faire le traitement.
Il y a deux conséquences à cela :

- Cela produit un shuffle assez important des données car il doit le faire pour chaque donnée de chaque partition du RDD. Donc, si les partitions ne sont pas sur une même machine, cela provoque un gros trafic réseau.
- Si dans notre exemple les trajets avaient lieu à 99% le samedi, groupByKey rassemblera toutes les données ayant comme clé "samedi" dans une seule machine, ce qui peut causer des problèmes de mémoire.

Pour `reduceByKey`cela se passe autrement. Il y a d'abord un prétraitement dans chaque partition. Ensuite les nouvelles clé/valeur sont déplacées dans les partitions selon leur clé pour avoir leur traitement final.
La quantité de données est donc moins élevée, réduisant ainsi le temps de traitement.

# Nombre de trajets effectués en fonction de la tranche horaire


on divisera les tranche horaire en fonction de l'heure de départ du taxi : 
- 00:00 - 04:00 -> tranche horaire 1
- 04:00 - 08:00 -> tranche horaire 2 
- 08:00 - 12:00 -> tranche horaire 3
- 12:00 - 16:00 -> tranche horaire 4
- 16:00 - 20:00 -> tranche horaire 5
- 20:00 - 23:59 -> tranche horaire 6





In [12]:
df_with_time_slice = df \
    .withColumn("pickup_datetime",to_timestamp(col("pickup_datetime"))) \
    .withColumn("hour_pickup", date_format(col("pickup_datetime"), "hh:mm")) \

In [13]:
df_with_time_slice.take(1)

[Row(id='id2875421', vendor_id='2', pickup_datetime=datetime.datetime(2016, 3, 14, 17, 24, 55), dropoff_datetime='2016-03-14 17:32:30', passenger_count='1', pickup_longitude='-73.982154846191406', pickup_latitude='40.767936706542969', dropoff_longitude='-73.964630126953125', dropoff_latitude='40.765602111816406', store_and_fwd_flag='N', trip_duration='455', hour_pickup='05:24')]

In [14]:
"00:00"<="03:00"<"04:00"

True

In [15]:
time_slice_column = when(("00:00"<=col("hour_pickup")) & (col("hour_pickup")<"04:00"), "slice_1") \
    .when(("04:00"<=col("hour_pickup")) & (col("hour_pickup")<"08:00"), "slice_2")\
    .when(("08:00"<=col("hour_pickup")) & (col("hour_pickup")<"12:00"), "slice_3")\
    .when(("12:00"<=col("hour_pickup")) & (col("hour_pickup")<"16:00"), "slice_4")\
    .when(("16:00"<=col("hour_pickup")) & (col("hour_pickup")<"20:00"), "slice_5")\
    .when(("20:00"<=col("hour_pickup")) & (col("hour_pickup")<="23:59"), "slice_6")\
    .otherwise("slice_unknown")

In [16]:
df_with_time_slice = df_with_time_slice.withColumn("time_slice",time_slice_column)

In [17]:
df_with_trip_per_time_slice = df_with_time_slice.rdd.map(lambda x : (x.time_slice,1)) \
    .reduceByKey(lambda x,y: x + y)

In [18]:
df_with_trip_per_time_slice.collect()

[('slice_2', 441346),
 ('slice_3', 587163),
 ('slice_1', 305014),
 ('slice_4', 125121)]

# Nombre de km par jour de la semaine 

In [19]:
df_with_distance_day_of_week = df \
    .withColumn("a", pow(sin(radians(col("dropoff_latitude") - col("pickup_latitude")) / 2), 2) + cos(radians(col("pickup_latitude"))) * cos(radians(col("dropoff_latitude"))) * pow(sin(radians(col("dropoff_longitude") - col("pickup_longitude")) / 2), 2)) \
    .withColumn("distance", asin(sqrt(col("a"))) * 2 * r) \
    .withColumn("pickup_datetime",to_timestamp(col("pickup_datetime"))) \
    .withColumn("week_day_number", date_format(col("pickup_datetime"), "u")) \
    .withColumn("week_day", date_format(col("pickup_datetime"), "E")) \

In [20]:
df_with_km_per_time_slice = df_with_distance_day_of_week.rdd.map(lambda x : (x.week_day,x.distance)) \
    .reduceByKey(lambda x,y: x + y)

In [21]:
df_with_km_per_time_slice.collect()

[('Wed', 702918.7950886373),
 ('Mon', 668482.1133133217),
 ('Fri', 758724.4713666884),
 ('Thu', 747677.6291327236),
 ('Tue', 678328.0950559068),
 ('Sun', 726453.2498305361),
 ('Sat', 736411.1316969573)]

In [22]:
spark.stop()